In [1]:
import pandas as pd

from data_processor.ChinaStockDownloader import ChinaStockDownloader

In [2]:
# dl = ChinaStockDownloader("2010-01-01", "2022-07-01", index="TICKER_LIST_590")

# df = dl.download_fundament(remain_statdate=True)

In [3]:
time_table = pd.read_csv("./data/590_fund_time.csv", index_col=0)
time_table.tic = time_table.tic.apply(lambda x: x[3:])
time_table

,tic,pubDate,statDate
0,002287,2010-04-22,2010-03-31
1,002287,2010-08-27,2010-06-30
2,002287,2010-10-26,2010-09-30
3,002287,2011-04-01,2010-12-31
4,002287,2011-04-22,2011-03-31
...,...,...,...
28905,000800,2021-04-30,2021-03-31
28906,000800,2021-08-28,2021-06-30
28907,000800,2021-10-30,2021-09-30
28908,000800,2022-03-31,2021-12-31


In [4]:
df = time_table.set_index('tic')
df

,pubDate,statDate
tic,,
002287,2010-04-22,2010-03-31
002287,2010-08-27,2010-06-30
002287,2010-10-26,2010-09-30
002287,2011-04-01,2010-12-31
002287,2011-04-22,2011-03-31
...,...,...
000800,2021-04-30,2021-03-31
000800,2021-08-28,2021-06-30
000800,2021-10-30,2021-09-30


In [5]:
fund_path = "./data/fundamental_of_592/"

In [15]:
sample_fund_df = pd.read_csv("./data/fundamental_of_592/000011.csv", index_col=0)
sample_fund_df.columns

Index(['date', 'grossMargin', 'revenue', 'inventory_days', 'receivable_days',
       'payable_days', 'debt_ratio', 'fixedasset', 'freecashflow',
       'cashtoemployee', 'netmargin', 'num_shareholder', 'net_equity'],
      dtype='object')

In [21]:
from pprint import pprint

In [28]:
tic_list = list(time_table.tic.unique())
tic_with_missing = 0
tic_with_complete_missing_col = 0
complete_missing_col = dict(
    zip(sample_fund_df.columns, [0 for i in range(len(sample_fund_df.columns))])
)

for tic in tic_list:
    complete_missing_flag = False
    fund_df = pd.read_csv(fund_path + tic + ".csv", index_col=0).iloc[8:]
    fund_df["tic"] = tic
    pub_d = df.loc[tic]["pubDate"].to_list()
    stat_d = df.loc[tic]["statDate"].to_list()
    trans_dict = dict(zip(stat_d, pub_d))
    fund_df.date = fund_df.date.map(trans_dict)
    if fund_df.isnull().any().any():
        tic_with_missing += 1
        print(f"\n{tic} has missing data in: ")
        for col in fund_df.columns:
            null_num = fund_df.isnull()[col].sum()
            if null_num > 0:
                print(f"{col}: {null_num}")
            if null_num == 48:
                complete_missing_flag = True
                complete_missing_col[col] += 1
        if complete_missing_flag:
            tic_with_complete_missing_col += 1

print(f"\nThere are {tic_with_missing} stocks have missing data")
print(
    f"{tic_with_complete_missing_col} stocks have columns that are missing completely.\n"
)
print(f"The number of complete missing column: ")
for key, value in complete_missing_col.items():
    print(f"{key}: {value}")



000543 has missing data in: 
inventory_days: 1

600131 has missing data in: 
receivable_days: 5

002245 has missing data in: 
receivable_days: 1

600809 has missing data in: 
cashtoemployee: 1

601328 has missing data in: 
grossMargin: 48
inventory_days: 48
receivable_days: 48
payable_days: 48
fixedasset: 48
freecashflow: 25
netmargin: 48

000898 has missing data in: 
receivable_days: 1

600012 has missing data in: 
receivable_days: 48

002024 has missing data in: 
receivable_days: 1

600188 has missing data in: 
receivable_days: 1

600403 has missing data in: 
grossMargin: 2
revenue: 2
inventory_days: 3
receivable_days: 2
payable_days: 2
fixedasset: 3

600036 has missing data in: 
grossMargin: 48
inventory_days: 48
receivable_days: 48
payable_days: 48
fixedasset: 48
freecashflow: 26
netmargin: 48

601808 has missing data in: 
receivable_days: 2

600605 has missing data in: 
inventory_days: 15

600287 has missing data in: 
payable_days: 1

600887 has missing data in: 
receivable_days:

In [16]:
for tic in tic_list[:3]:
    price_df = pd.read_csv(f"./data/590data/2010-2021_592_fq/{tic}.csv", index_col=0)
    fund_df = pd.read_csv(fund_path + tic + ".csv", index_col=0).iloc[8:]
    fund_df['tic'] = tic
    pub_d = df.loc[tic]['pubDate'].to_list()
    stat_d = df.loc[tic]['statDate'].to_list()
    trans_dict = dict(zip(stat_d, pub_d))
    fund_df.date = fund_df.date.map(trans_dict)
    
price_df.rename(columns={"Date": "date"}, inplace=True)

price_df.merge(fund_df, on="date")

,date,Open,Close,High,Low,Volume,grossMargin,revenue,inventory_days,receivable_days,payable_days,debt_ratio,fixedasset,freecashflow,cashtoemployee,netmargin,num_shareholder,net_equity,tic
0,2010-04-23,20.44,20.54,21.11,20.44,9812.39,4.9506,9.326452e+08,18.8025,36.3269,56.294050,50.3836,5.025582e+09,-8.282699e+08,6.567754e+07,3.3519,87785.0,5.404846e+09,000543
1,2010-08-24,17.82,17.86,17.98,17.40,8161.02,0.9232,7.628001e+08,21.4990,32.6477,57.422652,52.8075,5.014268e+09,-1.276295e+09,1.451534e+08,3.5228,83484.0,5.138724e+09,000543
2,2010-10-27,19.33,18.91,19.33,18.91,17711.46,0.1874,9.965623e+08,18.6303,35.1920,66.312966,54.6245,4.865571e+09,-1.397725e+09,2.184005e+08,4.0479,81990.0,5.123374e+09,000543
3,2011-03-18,18.24,18.47,18.61,18.24,13239.10,0.5659,1.092277e+09,13.9392,36.4897,52.127144,57.8041,4.923252e+09,-2.022087e+09,2.916614e+08,2.2791,78902.0,5.322664e+09,000543
4,2011-04-28,22.39,21.41,22.97,20.90,30280.62,-1.0792,9.730159e+08,14.7730,38.3142,44.424494,58.0940,4.848914e+09,-1.110184e+09,9.326512e+07,4.4916,70039.0,5.315397e+09,000543
5,2011-08-17,15.66,15.75,16.08,15.54,9664.56,-0.3129,1.047969e+09,15.7087,35.5492,41.472755,59.8437,7.844281e+09,-1.220345e+09,1.728313e+08,2.9680,60774.0,5.188093e+09,000543
6,2011-10-20,12.78,12.30,12.83,12.23,4019.26,0.7727,1.535185e+09,14.6661,37.3072,44.629699,62.0249,7.776433e+09,-1.667999e+09,2.501359e+08,0.6868,63012.0,5.073271e+09,000543
7,2012-03-23,12.41,12.04,12.44,12.02,2902.55,0.4191,1.594203e+09,15.2805,36.4306,56.157994,63.7156,7.946574e+09,-2.821300e+09,3.582876e+08,0.2755,67246.0,4.910198e+09,000543
8,2012-04-27,13.13,12.90,13.15,12.74,7939.78,9.0188,1.568837e+09,25.4806,35.1727,66.941868,66.7547,7.840040e+09,-2.155773e+09,9.865889e+07,-1.7634,66003.0,5.192649e+09,000543
9,2012-08-20,14.77,14.96,15.05,14.54,7273.54,8.4781,1.633490e+09,28.2668,39.6589,72.262518,66.4743,1.151805e+10,-6.117992e+08,1.826950e+08,-0.3606,60955.0,5.397309e+09,000543
